The Flight.jl package is built upon a hierarchical, causal modeling paradigm. At its core lies the parametric `System` type, which represents a physical system with continuous and (optionally) discrete dynamics.  Every `System` has a continuous state $x$. It may also have an external input $u$, an output $y$ and a discrete state $s$.

The `System`'s continuous dynamics are described by a differential equation of the form:
$$
\dot{x} = g(x, u, s, t)
$$

Its output is given by an algebraic equation of the form:
$$
y = h(x, u, s, t)
$$

And its discrete dynamics are described by a difference equation of the form:
$$
s_{k+1} = f_d(s_{k}, u_{k}, x_{k}, t_{k})
$$

A concrete `System` type is defined by a subtype of the abstract `SystemDescriptor` type. This `SystemDescriptor` serves a dual purpose:
1. It holds the values for all the (immutable) parameters that characterize a specific instance of the `System`.
2. It provides a dispatch mechanism for extending the set of functions that initialize the `System`'s variables: $x$, $u$, $y$ and $s$.

To clarify this, let us consider a simple example: a [mass-spring-damper][1] model.

This model is characterized by a mass $m$, a spring constant $k$ and a damping constant $c$. From these parameters, it is useful to define:
- The undamped natural frequency, $\omega_n = \sqrt{{k}/{m}}$
- The damping ratio, $\zeta = {c}/{2m\omega_n}$

The continuous dynamics of the mass-spring damper model, in state-space form, are given by:
$$
\dot{v} = f - \omega^2_n p - 2\zeta \omega_n v \\
\dot{p} = v
$$

Where $p$ denotes the mass' position, $v$ its velocity and $f = F/m$ is a mass-normalized, externally applied force.

The model's continuous state is $x = \begin{pmatrix} v & p \end{pmatrix}^T$. Its only external input is the normalized force $f$. And as outputs we will (arbitrarily) choose $p$, $v$ and the acceleration, $a = \dot{v}$.

In its vanilla version, the mass-spring-damper model does not have any discrete state variables. To illustrate a use case for $s$, let's add a twist to the model: the damper will be initially disengaged; we want it to engage only after the mass has been in motion for more than $\Delta t_d$ seconds, and disengage again whenever it has been stationary for more than $\Delta t_d$ seconds. This behavior can be implemented by a simple finite state machine that keeps track of three variables: the damper's state (engaged or disengaged), the last time it started moving, and the last time it stopped. These clearly represent states in our system, but unlike $x$, their evolution is not described by a differential equation. Although two of them are actually continuous variables, they are all updated at discrete time steps. Therefore, they belong in the discrete state $s$.

The vanilla model's dynamic response is entirely determined by the two constant parameters $\omega_n$ and $\zeta$. To these, we now need to add $\Delta t_d$, and a velocity threshold $v_{\epsilon}$ below which we consider the mass stationary. With this, we can define our `SystemDescriptor` subtype as:


[1]: https://en.wikipedia.org/wiki/Mass-spring-damper_model

In [1]:
using Flight

Base.@kwdef struct MassSpringDamper <: SystemDescriptor
    ω_n::Float64 = 1.0 #undamped natural frequency
    ζ::Float64 = 0.5 #damping ratio
    Δt_d::Float64 = 2 #spring switching time interval
    v_ϵ::Float64 = 1e-6 #velocity threshold
end

MassSpringDamper

In [ ]:

abs(v) > v_eps ? s.t_last_moving = t : s.t_last_stopped = t
Δt_stopped = t - t_last_moving
Δt_moving = t - t_last_stopped

if s.damper_engaged #s.damper_engaged #state 1
    #el damper se desenengancha cuando pasan mas de 2 segundos desde el ultimo
    #instante en que el sistema estuvo en movimiento
    Δt_stopped > Δt_damper ? s.damper_engaged = false : nothing
else #!s.damper_engaged #state 0
    #el damper se activa cuando han pasado mas de 2 segundos desde que el
    #sistema estuvo parado
    Δt_moving > Δt_damper ? s.damper_engaged = true : nothing
end

#cuando me pare, necesito empezar a contar cuanto tiempo llevo parado
    





    
# elses.sign_memory * sign_current < 0 ? counter += 1 : nothing
#     s.sign_memory = sign(v)
# end

# if s.sign_memory * sign(p) < 0
#     counter += 1
#     s.sign_memory = sign(p)
# end
# s.damper_engaged = (counter >= 4 ? true : false)
### add plots!


Now, to define the `System`'s $x$, $u$, $y$ and $s$, we need to follow a few rules:
1. $x$ must be a subtype of `AbstractArray{Float64}`
2. $u$ and $s$ can be any mutable types
3. $y$ must be an immutable type, and its fields must themselves be immutable as well

While $x$ can be defined as a plain `Vector{Float64}`, it is generally a better idea to use a labelled `ComponentVector{Float64}` (provided by the awesome [ComponentArrays.jl][ca] package). In fact, as we will see later, when multiple `System`s are composed, their respective continuous state vectors are automatically stacked as ```ComponentVector``` blocks.

$x$, $u$ and $s$ are preallocated upon initialization and modified in-place during simulation. In contrast, since $y$ is meant to be logged during simulation, a new instance must be created at every time step. Because mutable types are heap-allocated, making $y$ purely immutable is the price to pay to avoid performance-killing memory allocations.

With all of this in mind, we can define the `System`'s variables through the following initialization function extensions:

[ca]: https://github.com/jonniedie/ComponentArrays.jl

In [14]:
using ComponentArrays

Base.@kwdef struct MassSpringDamperY
    p::Float64 = 0.0
    v::Float64 = 0.0
    a::Float64 = 0.0
end

Systems.init(::SystemX, ::MassSpringDamper) = ComponentVector(p = 0.0, v = 0.0)
Systems.init(::SystemU, ::MassSpringDamper) = Ref(0.0)
Systems.init(::SystemY, ::MassSpringDamper) = MassSpringDamperY()
Systems.init(::SystemD, ::MassSpringDamper) = nothing

A few comments are in order:
- `SystemX`, `SystemU`, `SystemY` and `SystemD` are empty, trait-like types, used only for dispatch within the `System` constructor. 
- Our `System`'s input (the mass-normalized force $f$) is a scalar. Since a plain `Float64` would be immutable, in order to be able to modify its value during simulation, we wrap it around a `Ref`. Alternatively, we could define a custom mutable type wrapping a single `Float64`.
- Being immutable, we could also declare $y$ to be a `NamedTuple`, but defining a custom type is more convenient in certain scenarios.
- In this case, we are throwing away the `MassSpringDamper` argument, but if needed we could use it to make the initial values for the `System`'s variables depend on the values of the `MassSpringDamper` parameters.
- If our `System` had no $u$, $y$ or $s$, their corresponding method definitions could be omitted and they would be initialized to `nothing` by default. Not so $x$, which is always required.

We can already instantiate our `System`.

In [18]:
sys_desc = MassSpringDamper(ζ = 0.5) #let's make it underdamped
sys = System(MassSpringDamper())
@show typeof(sys)
Utils.showfields(sys) #show the type's fields with their current values

typeof(sys) = System{MassSpringDamper, ComponentVector{Float64, Vector{Float64}, Tuple{Axis{(p = 1, v = 2)}}}, MassSpringDamperY, Base.RefValue{Float64}, Nothing, NamedTuple{(:ω_n, :ζ), Tuple{Float64, Float64}}, NamedTuple{(), Tuple{}}}
ẋ: (p = 0.0, v = 0.0)
x: (p = 0.0, v = 0.0)
y: MassSpringDamperY(0.0, 0.0, 0.0)
u: Base.RefValue{Float64}(0.0)
d: nothing
t: Base.RefValue{Float64}(0.0)
params: (ω_n = 1.0, ζ = 0.5)
subsystems: NamedTuple()


We can see that all the `System`'s variables have been allocated as specified in our methods, and the parameters for this particular `MassSpringDamper` instance are now available to us in the `System`'s `params` field.

Now, the only missing piece is the `System`'s continuous and discrete dynamics. These are defined by two additional function extensions:

In [25]:
using UnPack

function Systems.f_cont!(sys::System{<:MassSpringDamper}) 
    
    @unpack x, u, params = sys #don't need s or t
    @unpack p, v = x
    @unpack ω_n, ζ = params

    f = u[] #de-reference to get the underlying Float64
    a = f - ω_n^2 * p - 2ζ * ω_n * v
    
    #update sys.ẋ
    sys.ẋ.v = a
    sys.ẋ.p = v

    #update sys.y (cannot be mutated, we need to assign a new instance to it)
    sys.y = MassSpringDamperY(; p, v, a) 
end

#no discrete dynamics
Systems.f_disc!(sys::System{<:MassSpringDamper}) = false


(Tip: $\dot{x}$ is typed as `x\dot`)

Function `f_cont!` packs the `System`'s continuous dynamics $\dot{x} = g(x,u,s,t)$ and observation equation $y = h(x,u,s,t)$ into a single equation $(\dot{x}, y) = f(x, u, s, t)$. This is a design decision. It is motivated by the fact that, for many complex `System`s in this package, $\dot{x}$ and $y$ share lots of intermediate computations, which would have to be performed twice if separate methods were defined for $g$ and $h$.

Function `f_disc!` implements the discrete dynamics function $x_{k+1} = f_d(x_{k}, u_{k}, s_{k}, t_{k})$. Therefore, in theory it should only mutate the discrete state $s$. However, in scenarios it may be useful to modify $x$ too at certain discrete epochs (an example is the periodic renormalization of unit quaternions or unit vectors in $x$ to correct for deviations from their unit norm constraint due to  numerical error). The `Bool` return value of `f_disc!` declares whether it has modified $x$. This flag is in turn passed during simulation to the `OrdinaryDiffEq` integrator so it can handle the discontinuity.

For safety reasons, no trivial fallback methods are provided for `f_cont!` and `f_disc!`. They must be extended for every `System`, even if it has no continuous or discrete dynamics. This is how to extend them in a trivial case:

In [21]:
struct NoContinuousDynamics <: SystemDescriptor end
struct NoDiscreteDynamics <: SystemDescriptor end

Systems.f_cont!(::System{NoContinuousDynamics}, args...; kwargs...) = nothing
Systems.f_disc!(::System{NoDiscreteDynamics}, args...; kwargs...) = false


Let's do a quick check to confirm our methods do not allocate:

In [26]:
using BenchmarkTools

@btime (f_cont!($sys))
@btime (f_disc!($sys))


  2.600 ns (0 allocations: 0 bytes)
  0.001 ns (0 allocations: 0 bytes)


false

The `Simulation` interface simply extends a few `ODEIntegrator` functions which are useful in this context: `step!`, reinit!, add_tstop! and get_proposed_dt.

When a simulation step is taken, the `Simulation` calls its internal `ODEIntegrator` to update the underlying `System`'s $x$ and $y$ from $t_k$ to $t_{k+1}. This typically involves multiple calls to `f_cont!`, depending on the chosen ODE solver. Then, `f_disc!` is called once to update $s_{k}$ to $s_{k+1}$. During this call, `f_disc!` might modify the already updated $x_{k+1}$ (in which case it must return `true`). These changes in `f_disc!` will not propagate to $y$ until `f_cont!` is called again on the next simulation step.


Since f_disc! is run after each step taken by the ODEIntegrator, if the discrete dynamics need to be updated with a fixed period (as will often be the case), a fixed step ODE solver must be chosen.

In [ ]:

discrete state: engaged/disengaged at 5 secs intervals. if t is between 2kT and (2k+1), engaged. otherwise, disengage


Arbitrarily complex systems with continuous and discrete dynamics can be assembled from simpler subsystems. 